IMPORTING REQUIRED PACKAGES

In [3]:
#import cv2
#import mediapipe
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


In [1]:
#creating a dictionary of all the 5 yoga poses

POSE_DICT = {
    "downdog": 0,
    "goddess":1,       
    "plank":2,
    "tree":3,
    "warrior2":4
}
POSE_DICT



{'downdog': 0, 'goddess': 1, 'plank': 2, 'tree': 3, 'warrior2': 4}

In [2]:
#creating a list of all the keypoints extracted (33)

key_points = ['Nose','Left_eye_inner','Left_eye','Left_eye_outer','right_eye_inner',
              'right_eye','right_eye_outer','left_ear','right_ear','mouth_left',
              'mouth_right','lef_shoulder','right_shoulder','lef_elbow','right_elbow',
              'left_wrist','right_wrist','left_pinky','right_pinky','left_index',
              'right_index','left_thumb','right_thumb','left_hip','right_hip',
              'left_knee','right_knee','left_ankle','right_ankle',
              'left_heel','right_heel','left_foot_index','right_foot_index']

In [8]:
#creating an empty dictionary to add all the 5 csv files for TRAIN DATASET 
data_train = {}

#loop to iterate over all the items of dictionary and append the csv files on it
for pose, class_id in POSE_DICT.items():
    data_train[class_id] = pd.read_csv(os.path.join('C:\\Users\\HBD\\Documents\\YogaPose_classification\\TrainDS\\', '{}.csv'.format(pose)), index_col=0)

In [67]:
#creating an empty dictionary to add all the 5 csv files for TRAIN DATASET 
data_test = {}

#loop to iterate over all the items of dictionary and append the csv files on it
for pose, class_id in POSE_DICT.items():
    data_test[class_id] = pd.read_csv(os.path.join('C:\\Users\\HBD\\Documents\\YogaPose_classification\\TestDS\\', '{}.csv'.format(pose)), index_col=0)

In [171]:
#printing entries to check if its appened correctly on train dataset dictionary
print(data_train[0][['Nose y', 'right_wrist y', 'left_wrist y','pose']].head())
print(data_train[1][['Nose y', 'right_wrist y', 'left_wrist y','pose']].head())

In [68]:
#printing entries to check if its appened correctly on test dataset dictionary
print(data_test[0][['Nose y', 'right_wrist y', 'left_wrist y','pose']].head())
print(data_test[1][['Nose y', 'right_wrist y', 'left_wrist y','pose']].head())

     Nose y  right_wrist y  left_wrist y     pose
0  0.759728       0.896647      0.846163  downdog
1  0.764660       0.872627      0.910119  downdog
2  0.739474       0.853756      0.928014  downdog
3  0.684020       0.830864      0.899128  downdog
4  0.645560       0.725713      0.751534  downdog
     Nose y  right_wrist y  left_wrist y     pose
0  0.251286       0.111901      0.106405  goddess
1  0.367380       0.377851      0.377205  goddess
2  0.359367       0.341215      0.334023  goddess
3  0.453918       0.555632      0.676420  goddess
4  0.357837       0.338963      0.339704  goddess


NOW THE ABOVE CSV FILES HAVE TO BE COMBINED IN 2 SEPERATE CSV FILES. ONE FOR TRAIN AND ONE FOR TEST DATA.

In [21]:
#address of the directory where the 5 csv files for TRAIN dataset are saved
os.chdir("C:\\Users\\HBD\\Documents\\YogaPose_classification\\TrainDS")

In [22]:
extension_train = 'csv'

#listing all the files in that directory with extention 'csv'
all_filenames_train = [i for i in glob.glob('*.{}'.format(extension_train))]


In [23]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames_train ])

#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [71]:
#address of the directory where the 5 csv files for TEST dataset are saved
os.chdir("C:\\Users\\HBD\\Documents\\YogaPose_classification\\TestDS")

extension_test = 'csv'

all_filenames_test = [i for i in glob.glob('*.{}'.format(extension_test))]


In [72]:
#combine all files in the list
test_combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames_test ])
#export to csv
test_combined_csv.to_csv( "test_combined_csv.csv", index=False, encoding='utf-8-sig')

READING THE TRAIN AND TEST CSV FILES INTO DATAFRAME 

In [148]:
train_df = pd.read_csv("C:\\Users\\HBD\\Documents\\YogaPose_classification\\TrainDS\\combined_csv.csv")

In [73]:
test_df = pd.read_csv("C:\\Users\\HBD\\Documents\\YogaPose_classification\\TestDS\\test_combined_csv.csv")

In [135]:
train_df.head(2)

,pose,Nose x,Nose y,Left_eye_inner x,Left_eye_inner y,Left_eye x,Left_eye y,Left_eye_outer x,Left_eye_outer y,right_eye_inner x,...,right_ankle x,right_ankle y,left_heel x,left_heel y,right_heel x,right_heel y,left_foot_index x,left_foot_index y,right_foot_index x,right_foot_index y
0,downdog,0.473025,0.724765,0.456647,0.730075,0.454467,0.728076,0.452063,0.725779,0.455921,...,0.784478,0.743638,0.82828,0.776433,0.801361,0.759982,0.745398,0.794812,0.736808,0.784433
1,downdog,0.563061,0.657513,0.580647,0.663995,0.582510,0.661348,0.584492,0.658282,0.580242,...,0.258608,0.739572,0.24691,0.782398,0.234630,0.796620,0.332900,0.804314,0.325635,0.825382


In [136]:
test_df.head(2)

,pose,Nose x,Nose y,Left_eye_inner x,Left_eye_inner y,Left_eye x,Left_eye y,Left_eye_outer x,Left_eye_outer y,right_eye_inner x,...,right_ankle x,right_ankle y,left_heel x,left_heel y,right_heel x,right_heel y,left_foot_index x,left_foot_index y,right_foot_index x,right_foot_index y
0,downdog,0.544188,0.759728,0.564265,0.777088,0.566724,0.775801,0.569913,0.774041,0.564282,...,0.183566,0.817694,0.168044,0.843284,0.153090,0.848239,0.270753,0.878541,0.262585,0.883095
1,downdog,0.378873,0.764660,0.355602,0.755793,0.354001,0.750477,0.352241,0.744647,0.355824,...,0.876976,0.821419,0.940832,0.901670,0.916826,0.876229,0.784191,0.935821,0.774428,0.914212


In [169]:
train_df = train_df.drop(['Unnamed: 0'], axis=1)


In [76]:
test_df = test_df.drop(['Unnamed: 0'], axis=1)

In [134]:
#test_df.head(2)

SEPERATING THE DEPENDENT AND INDEPENDENT FEATURES FROM TRAIN DATASET

In [161]:
X = train_df.drop('pose',axis=1)
y = train_df['pose']

 SPLITING THE TRAIN DATASET 

In [162]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [163]:
#all the values were already scaled down while saving into csv files

#sc= StandardScaler().fit(X_train)
#X_train_std = sc.transform(X_train)
#X_valid_std = sc.transform(X_valid)
#X_valid_std

# SVM MODEL 

In [164]:
svc=SVC() #Default hyperparameters

#fitting the model
svc.fit(X_train,y_train)

#predicting the result
y_valid_pred = svc.predict(X_valid)
y_train_pred = svc.predict(X_train)

print('Test Accuracy Score:')
print(metrics.accuracy_score(y_valid,y_valid_pred))

Test Accuracy Score:
0.8431372549019608


In [165]:
print(classification_report(y_valid, y_valid_pred))

              precision    recall  f1-score   support

     downdog       1.00      0.86      0.92         7
     goddess       0.70      0.78      0.74         9
       plank       1.00      0.92      0.96        12
        tree       0.93      0.88      0.90        16
    warrior2       0.56      0.71      0.63         7

    accuracy                           0.84        51
   macro avg       0.84      0.83      0.83        51
weighted avg       0.87      0.84      0.85        51



THE ACCURACY FOR DEFAULT SVM MODEL CAME 84% ON THE VALIDATION DATA. FROM THE ABOVE REPORT WE CAN SEE THAT THE MODEL SHOWS COMPARATIVELY LESS ACCURACY FOR WARRIOR2 IMAGES. IT IS BECAUSE THE IMAGES HAVE MANY VARIATIONS IN TERMS OF POSITION OF KEYPOINTS I.E. THERE ARE A LOT OF DIFFERENT POSES FOR WARRIOR2 YOGA POSE AND THE REST OF THE POSES HAVE VERY SIMILAR KIND OF IMAGES SO ITS EASIER TO DETECT AND CLASSIFY THEM.

# Hypertuning SVM model

In [166]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train,y_train)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 2/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 3/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 4/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 5/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV 1/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 2/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 3/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 4/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 5/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV 1/5] END ..................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV 2/5] END ..................C=0.1, gamma=0.0

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [167]:
#best parameter after tuning
print(grid.best_params_)

# printing how  model looks after hyper-parameter tuning
print(grid.best_estimator_)


{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
SVC(C=100, gamma=0.1)


In [168]:
#making the predictions using best estimator
grid_predictions = grid.best_estimator_.predict(X_valid)

# print classification report
print(classification_report(y_valid, grid_predictions))

              precision    recall  f1-score   support

     downdog       0.88      1.00      0.93         7
     goddess       0.73      0.89      0.80         9
       plank       1.00      0.92      0.96        12
        tree       1.00      0.88      0.93        16
    warrior2       1.00      1.00      1.00         7

    accuracy                           0.92        51
   macro avg       0.92      0.94      0.92        51
weighted avg       0.93      0.92      0.92        51



THE ACCURACY HAS INCREACED AFTER HYPERPARAMETER TUNING TO 92%

USING THE TEST DATASET TO CHECK THE ACCURACY OF SVM MODEL

In [73]:
test_df = pd.read_csv("C:\\Users\\HBD\\Documents\\YogaPose_classification\\TestDS\\test_combined_csv.csv")

In [113]:
#seperating the independent features(keypoints) and target feature(pose)
L = test_df.drop('pose',axis=1)
m = test_df['pose']

In [172]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [173]:
X_train =  X
X_test = L
y_train = y
y_test = m

In [174]:
svc=SVC() #Default hyperparameters

svc.fit(X_train,y_train)

y_test_pred = svc.predict(X_test)

print('Test Accuracy Score:')
print(metrics.accuracy_score(y_test,y_test_pred))

Test Accuracy Score:
0.9191919191919192


In [118]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

     downdog       0.94      1.00      0.97        15
     goddess       0.92      0.69      0.79        16
       plank       0.96      0.96      0.96        27
        tree       0.92      0.96      0.94        24
    warrior2       0.84      0.94      0.89        17

    accuracy                           0.92        99
   macro avg       0.92      0.91      0.91        99
weighted avg       0.92      0.92      0.92        99



THE ACCURACY ON TEST DATASET WITHOUT HYPER PARAMETER TUNING COMES OUT TO BE 92%

In [125]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)


{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
SVC(C=100, gamma=0.1)


In [123]:
#predicting on best estimators after hyperparemeter tuning of SVM
grid_predictions = grid.best_estimator_.predict(X_test)

# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

     downdog       0.94      1.00      0.97        15
     goddess       0.92      0.75      0.83        16
       plank       0.96      0.96      0.96        27
        tree       0.92      0.92      0.92        24
    warrior2       0.89      1.00      0.94        17

    accuracy                           0.93        99
   macro avg       0.93      0.93      0.92        99
weighted avg       0.93      0.93      0.93        99



THE ACCURACY REMAINS SAME i.e. 93%

# RANDOM FOREST MODEL

In [175]:
rf=RandomForestClassifier(n_estimators=500,random_state = 0)
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)

In [176]:
# print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     downdog       0.94      1.00      0.97        15
     goddess       0.79      0.69      0.73        16
       plank       0.96      0.96      0.96        27
        tree       1.00      0.96      0.98        24
    warrior2       0.74      0.82      0.78        17

    accuracy                           0.90        99
   macro avg       0.88      0.89      0.88        99
weighted avg       0.90      0.90      0.90        99



TUNING THE RF CLASSIFIER

In [177]:
rf = RandomForestClassifier(max_features='auto', oob_score=True, random_state=1, n_jobs=-1)

param_grid = {'max_depth': [5, 10, 20],              
              "criterion" : ["gini", "entropy"], 
              "min_samples_leaf" : [1, 2], 
              "min_samples_split" : [5, 10], 
              "n_estimators" : [100,200,300]}

grid = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1, verbose = 3)

#fitting the model with train dataset
grid.fit(X_train,y_train)


Fitting 2 folds for each of 72 candidates, totalling 144 fits


GridSearchCV(cv=2,
             estimator=RandomForestClassifier(n_jobs=-1, oob_score=True,
                                              random_state=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 20], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=3)

In [178]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)


{'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200,
                       n_jobs=-1, oob_score=True, random_state=1)


In [105]:
grid_predictions = grid.best_estimator_.predict(X_test)

# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

     downdog       0.94      1.00      0.97        15
     goddess       0.92      0.69      0.79        16
       plank       0.96      0.96      0.96        27
        tree       1.00      0.96      0.98        24
    warrior2       0.76      0.94      0.84        17

    accuracy                           0.92        99
   macro avg       0.92      0.91      0.91        99
weighted avg       0.93      0.92      0.92        99



THE ACCURACY CAME OUT TO BE 92%. HENCE BOTH THE MODELS AFTER TUNING GIVES SAME ACCURACY IN PREDICTION ON TEST DATASET.
BUT SVM GIVES BETTER PRECISION SCORE ESPECIALLY FOR WARRIOR2 POSE IMAGES (.89) WHERE AS RF GIVES 0.76. THAT MEANS THAT SVM IS CORRECTLY CLASSIFYING POSITIVE OBSERVATIONS WITH RESPECT TO THE TOTAL PREDICTED POSITIVE OBSERVATIONS.  